#### Loading libraries

In [1]:
# Load tensorflow
import tensorflow as tf
# Below command is to avoid the known bug which prevents computation on some GPU devices
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)
# Load preprocessing tools
from scipy.ndimage.filters import gaussian_filter
from tensorflow.keras.utils import to_categorical
from PIL import Image
# Load model building blocks
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
# Load pre-trained model library
from tensorflow.keras import applications
# Load miscelaneous libraries
import numpy as np
import os
import matplotlib.pylab as plt
import gc
# Settings to set the max output of numpy to infty
# import sys
# np.set_printoptions(threshold=sys.maxsize)

#### Define image processor and data generator

In [2]:
# Define gaussian blur class
class GaussBlur:
    def __init__(self, radius):
        self.radius = radius
    def blur(self, image):
        return gaussian_filter(image, sigma = self.radius)

In [3]:
# Define class for data generation
class DataGenerator:
    'Generates data for Keras'
    def __init__(self, list_image_paths = None,  
                 dim = None,
                 n_channels = 3, 
                 rescale = 1, 
                 preprocessing_func = None):
        'Initialization'
        self.list_image_paths = list_image_paths
        self.dim = dim
        self.n_channels = n_channels
        self.rescale = rescale
        self.preprocessing_func = preprocessing_func

    def generate(self):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        
        # Initialisation
        X = np.empty((len(self.list_image_paths), *self.dim, self.n_channels))
        
        # Generate data
        for i, image_path in enumerate(self.list_image_paths):
            # Load image and transform
            image = Image.open(os.path.join(image_path))
            if self.dim is not None:
                image = image.resize(self.dim, resample = Image.NEAREST)
            image = np.array(image)[:, :, :self.n_channels]
            image = image * self.rescale
            if self.preprocessing_func is not None:
                image = self.preprocessing_func(image)
            # Store sample
            X[i,] = image

        return X

In [4]:
# Define sorter of image names in order by image number (default is alphanumric)
def sorter(item):
    # Since highest marks first, least error = most marks
    radius = float(item[1 : item.find('_')])
    num_img = int(item[item.find('g') + 1 : item.find('j') - 1])
    return (radius, num_img)

#### Load the data

In [5]:
# Extract image paths
img_paths = {}
for k in range(5):
    i = k + 2
    path_prefix = os.path.join(os.getcwd(), 'image_data', 'small', 'test', str(i))
    img_paths[i] = [os.path.join(path_prefix, name) for name in sorted(os.listdir(path_prefix), key = sorter)]

# Apply data generators
gaussblur = GaussBlur(1)
params = {'dim': (128, 128), 
          'n_channels': 3, 
          'rescale': 1 / 255, 
          'preprocessing_func': gaussblur.blur
          }
test_generator = {}
for k in range(5):
    i = k + 2
    test_generator[i] = DataGenerator(img_paths[i], **params)
    
# Load images (for all number of forces) as matrix
X = dict()
for k in range(5):
    i = k + 2
    X[i] = test_generator[i].generate()

In [6]:
# Load angle labels
path_prefix = os.path.join(os.getcwd(), 'labels', 'small', 'test')
y_lai = dict()
for k in range(5):
    i = k + 2
    path = os.path.join(path_prefix, str(i))
    y_lai[i] = np.load(os.path.join(path,'mags.npy'))

#### Load model

In [9]:
# Load models
models_path = os.path.join(os.getcwd(), 'models', 'small', '400')
model_class = load_model(os.path.join(models_path, 'vgg19_num_forces.h5'))                       

In [10]:
# predict class
predicted_class = dict()
indices_correct_class = dict()
indices_incorrect_class = dict()
predicted_incorrect_class = dict()
for k in range(5):
    i = k + 2
    predicted_class[i] = model_class.predict(X[i]).argmax(-1)+2
    indices_correct_class[i] = np.argwhere(predicted_class[i] == i).flatten().tolist()
    indices_incorrect_class[i] = np.argwhere(predicted_class[i] != i).flatten().tolist()
    predicted_incorrect_class[i] = predicted_class[i][indices_incorrect_class[i]]

In [14]:
models_m = dict()                         
for k in range(5):
    i = k + 2
    models_m[i] = load_model(os.path.join(models_path, 'InceptionResNetV2_mags_'+str(i)+'.h5'))
    models_m[i].compile(metrics = ['mean_absolute_error', "mean_absolute_percentage_error"])

In [15]:
for k in range(5):
    i = k + 2
    print("Num forces: ", i)
    loss = models_m[i].evaluate(X[i][indices_correct_class[i], ], y_lai[i][indices_correct_class[i], ])

Num forces:  2
63/63 [==============================] - 3s 43ms/step - loss: 0.0000e+00 - mean_absolute_error: 0.0175 - mean_absolute_percentage_error: 4.1401
Num forces:  3
63/63 [==============================] - 128s 2s/step - loss: 0.0000e+00 - mean_absolute_error: 0.0388 - mean_absolute_percentage_error: 11.2406
Num forces:  4
62/62 [==============================] - 133s 2s/step - loss: 0.0000e+00 - mean_absolute_error: 0.0565 - mean_absolute_percentage_error: 24.8542
Num forces:  5
61/61 [==============================] - 132s 2s/step - loss: 0.0000e+00 - mean_absolute_error: 0.0497 - mean_absolute_percentage_error: 19.3003
Num forces:  6
61/61 [==============================] - 142s 2s/step - loss: 0.0000e+00 - mean_absolute_error: 0.0422 - mean_absolute_percentage_error: 31.6270


In [32]:
diffs = []
for k in range(5):
    i = k + 2
    print("Num forces: ", i)
    if (len(indices_incorrect_class[i]) > 0):
        for j in range(len(indices_incorrect_class[i])):
            predicted_mags = models_m[predicted_incorrect_class[i][j]].predict(X[i][None, indices_incorrect_class[i][j], ])
            true_mags = y_lai[i][indices_incorrect_class[i][j], ]
            diff = abs(np.sum(predicted_mags) - np.sum(true_mags))
            diffs.append(diff)
            print("Predicted: ", predicted_mags)
            print("True: ", true_mags)
            print("Difference: ", diff)      

Num forces:  2
Predicted:  [[0.46800807 0.49000818 0.7402678 ]]
True:  [0.65301849 0.65301849]
Difference:  0.3922471652023938
Predicted:  [[0.44582164 0.3429497  0.24587663]]
True:  [0.45943544 0.45943544]
Difference:  0.11577706969777468
Predicted:  [[0.572962   0.45535553 0.8732518 ]]
True:  [0.88693319 0.88693319]
Difference:  0.1277028720525628
Predicted:  [[0.79643816 0.52738935 0.66079783]]
True:  [0.88693319 0.88693319]
Difference:  0.21075896431483088
Predicted:  [[0.6259276  0.5400902  0.97175837]]
True:  [0.98576964 0.98576964]
Difference:  0.16623684749835554
Predicted:  [[0.7300502 0.5354796 0.9390116]]
True:  [0.98576964 0.98576964]
Difference:  0.2330021558784825
Predicted:  [[0.42308518 0.6925765  0.6463727 ]]
True:  [0.98576964 0.98576964]
Difference:  0.20950487270122942
Predicted:  [[0.8149892 0.5400071 0.6476537]]
True:  [0.98576964 0.98576964]
Difference:  0.031110733606753982
Predicted:  [[0.8270701  0.54394853 0.77725816]]
True:  [0.98576964 0.98576964]
Differenc

Predicted:  [[1.0210313  0.7343537  0.72608143 0.78306466 0.84370404]]
True:  [1.23432569 0.64063127 0.69652787 0.86167246]
Difference:  0.6750775994409763
Num forces:  5
Predicted:  [[0.26417112 0.18351965 0.18298429 0.23324116]]
True:  [0.2111022  0.25953472 0.1827523  0.19624681 0.21504882]
Difference:  0.20076861713548166
Predicted:  [[0.6499755  0.6374639  0.7492618  0.6582171  0.62537354 0.87824374]]
True:  [0.6817882  0.61520382 0.5410805  0.51032546 0.90646298]
Difference:  0.9436744833785022
Predicted:  [[0.6380927  0.6221849  0.72953653 0.6136756  0.65329254 0.83466274]]
True:  [0.6817882  0.61520382 0.5410805  0.51032546 0.90646298]
Difference:  0.8365840102034534
Predicted:  [[0.6662591 0.6309574 0.730613  0.6506946 0.6565854 0.8000173]]
True:  [0.6817882  0.61520382 0.5410805  0.51032546 0.90646298]
Difference:  0.8802661085921253
Predicted:  [[0.6957407  0.63152975 0.64428777 0.63944924 0.6720623  0.54461014]]
True:  [0.90646298 0.6817882  0.61520382 0.5410805  0.51032546

Predicted:  [[0.6833255  0.57761526 0.59052277 0.60091174]]
True:  [0.32135601 0.67166858 0.64263235 0.55298259 0.73569811]
Difference:  0.47196223634293855
Predicted:  [[0.6323856  0.51494324 0.57375765 0.62433535]]
True:  [0.32135601 0.67166858 0.64263235 0.55298259 0.73569811]
Difference:  0.5789158572535831
Predicted:  [[0.53920245 0.6185595  0.50285923 0.68512547]]
True:  [0.73569811 0.32135601 0.67166858 0.64263235 0.55298259]
Difference:  0.5785911311488467
Predicted:  [[0.53634673 0.5882843  0.52903473 0.66657776]]
True:  [0.73569811 0.32135601 0.67166858 0.64263235 0.55298259]
Difference:  0.6040940512996036
Predicted:  [[0.6298859  0.4093064  0.44091254 0.55967736]]
True:  [0.71607069 0.59898762 0.45300378 0.71562338 0.67477916]
Difference:  1.1186823407465365
Predicted:  [[0.5413011  0.6694339  0.6278794  0.6698585  0.56973845 0.6325194 ]]
True:  [0.56078327 0.48578114 0.6992712  0.78487041 1.04829058]
Difference:  0.13173419982536272
Predicted:  [[0.5295186  0.6879561  0.64

Predicted:  [[0.7718573 0.6308644 0.7405001 0.6391189 0.9684611]]
True:  [0.66119285 1.11634767 0.74881275 0.74679122 0.82000799 1.0687498 ]
Difference:  1.4111004888661878
Predicted:  [[0.83890885 0.5969009  0.76588273 0.6244531  0.90584844]]
True:  [0.66119285 1.11634767 0.74881275 0.74679122 0.82000799 1.0687498 ]
Difference:  1.4299081384786145
Predicted:  [[0.8146214  0.6022716  0.73049587 0.63574636 0.93750787]]
True:  [0.66119285 1.11634767 0.74881275 0.74679122 0.82000799 1.0687498 ]
Difference:  1.4412590086110608
Predicted:  [[0.8152049  0.6242931  0.7159047  0.6177676  0.98126346]]
True:  [0.66119285 1.11634767 0.74881275 0.74679122 0.82000799 1.0687498 ]
Difference:  1.4074686586507337
Predicted:  [[0.80130714 0.6196716  0.7331274  0.6588217  0.7799306 ]]
True:  [0.66119285 1.11634767 0.74881275 0.74679122 0.82000799 1.0687498 ]
Difference:  1.5690439760335462
Predicted:  [[0.8779057  0.6430152  0.7285341  0.6626678  0.84081227]]
True:  [0.66119285 1.11634767 0.74881275 0.7

In [33]:
print("Average difference: ", sum(diffs) / len(diffs))

Average difference:  0.6271770325060653
